In [19]:
import pandas as pd
import glob
import json
import geopandas
import geocoder
import folium
import json
import ast
import re
import numpy as np
import matplotlib.pyplot as plt

from geopy.geocoders import Nominatim
from shapely.geometry import Polygon, LineString, Point

In [5]:
offers_df = pd.read_csv('all_data\oferta_eksport_final.csv')
pracuj_df = pd.read_csv('all_data\pracuj_inferred_location.csv')

### Mergowanie datasetów

In [62]:
offers_df.head()

,position,location,contract,employer,starting_from,address,open_street_raw
0,Pomocniczy robotnik drogowy,"05-622 Odrzywołek 8a, powiat: grójecki, woj: ...",Umowa o pracę na czas określony,POWIATOWY ZARZĄD DRÓG W GRÓJCU.,04.01.2022,05-622 Odrzywołek 8a,"{'place_id': 30857090, 'licence': 'Data © Open..."
1,Pomocniczy robotnik drogowy,"05-622 Odrzywołek 8a, powiat: grójecki, woj: ...",Umowa o pracę na czas określony,POWIATOWY ZARZĄD DRÓG W GRÓJCU.,04.01.2022,05-622 Odrzywołek 8a,"{'place_id': 30857090, 'licence': 'Data © Open..."
2,Betoniarz-zbrojarz,"Ujście, powiat: pilski, woj: wielkopolskie",Umowa o pracę na okres próbny,kontakt przez PUP,05.01.2022,Ujście,"{'place_id': 282527944, 'licence': 'Data © Ope..."
3,Spawacz (nr 0039),"Bydgoska 2D, 89-200 Zamość, powiat: nakielski...",Umowa zlecenie / Umowa o świadczenie usług,Warsztat Mechaniczny Maszyn Rolniczych i Ślus...,05.01.2022,Bydgoska 2D,"{'place_id': 321954131, 'licence': 'Data © Ope..."
4,stróż (nr 0038),"85-001 Bydgoszcz, powiat: m. Bydgoszcz, woj: ...",Umowa o pracę na czas określony,SOLID SECURITY SPÓŁKA Z OGRANICZONĄ ODPOWIEDZ...,05.01.2022,85-001 Bydgoszcz,"{'place_id': 282271219, 'licence': 'Data © Ope..."
...,...,...,...,...,...,...,...
17180,monter stolarki okiennej i drzwiowej,"Pszczyna, powiat: pszczyński, woj: śląskie",Umowa o pracę na czas określony,kontakt przez PUP,02.12.2016,Pszczyna,"{'place_id': 282381696, 'licence': 'Data © Ope..."
17181,montażysta elewacji wentylowanej,"Pszczyna, powiat: pszczyński, woj: śląskie",Umowa o pracę na czas określony,kontakt przez PUP,13.01.2016,Pszczyna,"{'place_id': 282381696, 'licence': 'Data © Ope..."
17182,prace interwencyjne-stolarz meblowy/montażysta...,"Ruda Śląska, powiat: m. Ruda Śląska, woj: ślą...",Umowa o pracę na czas określony,kontakt przez PUP,05.09.2017,Ruda Śląska,"{'place_id': 282341375, 'licence': 'Data © Ope..."
17183,blacharz samochodowy,"Studzionka, powiat: pszczyński, woj: śląskie",Umowa o pracę na czas określony,kontakt przez PUP,07.01.2016,Studzionka,"{'place_id': 283020402, 'licence': 'Data © Ope..."


In [76]:
pracuj_df.drop_duplicates()

,url,work_modes,remote,employment_type,schedule,contract,locations,location_url,duties,requirements,offered,about_us,salary,benefits,inferred_location
0,https://www.pracuj.pl/praca/inventory-manager-...,NaN,remote recruitment,team manager,full-time,contract of employment,"['Kraków', 'Kraków, Lesser Poland']",NaN,['Analyze data to provide visibility of missed...,['University degree in a technical or analytic...,['Chance to become a part of dynamically growi...,zooplus Polska Sp. z o.o.\nNumber 1 in Europe!...,NaN,NaN,"{'x': 19.9420309, 'y': 50.0636007, 'addr:stree..."
1,https://www.pracuj.pl/praca/financial-analyst-...,NaN,remote recruitment,"specialist (mid / regular), senior specialist ...",full-time,contract of employment,"['Warszawa', 'Warszawa, Masovian']",NaN,['Support in planning (business plan and rolli...,"['University degree with Controlling, Finance,...",['Exciting work in an international environmen...,Technicolor Polska Sp. z o.o.\nOur Strength is...,NaN,NaN,"{'x': 21.071432235636493, 'y': 52.2337172, 'ad..."
2,https://www.pracuj.pl/praca/gl-accountant-wars...,NaN,remote recruitment,"specialist (mid / regular), senior specialist ...",full-time,contract of employment,"['Warszawa', 'Warszawa, Masovian']",NaN,['Performing general accounting and reporting ...,['Professional experience in the area of accou...,['Exciting work in an international environmen...,Technicolor Polska Sp. z o.o.\nOur Strength is...,NaN,NaN,"{'x': 21.071432235636493, 'y': 52.2337172, 'ad..."
3,https://www.pracuj.pl/praca/monter-urzadzen-el...,NaN,rekrutacja zdalna,specjalista (mid / regular),pełny etat,umowa o pracę,"['aleja Grunwaldzka 103a, Gdańsk', 'Gdańsk, po...",https://www.google.pl/maps/search/54.379585266...,['Wykonywanie prac elektrycznych zgodnie z otr...,['Wykształcenie techniczne - średnie lub zawod...,[],MACGREGOR POLAND SP Z O O\nMacGregor shapes th...,NaN,NaN,"{'x': '18.605209350585938', 'y': '54.379585266..."
4,https://www.pracuj.pl/praca/operator-maszyn-i-...,praca stacjonarna,NaN,pracownik fizyczny,pełny etat,umowa o pracę,"['Trzciniec (pow. bydgoski)', 'Trzciniec (pow....",NaN,['Bieżąca obsługa maszyn i urządzeń sterowanyc...,"['Wykształcenie min. zawodowe techniczne,', 'D...",['Pracę w stabilnej firmie o ugruntowanej pozy...,Xella Polska Sp. z o.o.\nXella Polska jest lid...,NaN,NaN,"{'x': 17.9480419, 'y': 53.0858024, 'addr:stree..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39441,https://www.pracuj.pl/praca/kierowca-miedzynar...,NaN,NaN,pracownik fizyczny,pełny etat,umowa o pracę,"['Tychy', 'Tychy, śląskie']",NaN,"['KIEROWCA C+E MIĘDZYNARODÓWKA', 'DOKUPILIŚMY ...",['minimum pół roku doświadczenia w pracy jako ...,['legalne zatrudnienie w oparciu o umowę o pra...,NaN,8 000‐8 500 zł brutto / mies.,8 000‐8 500 zł brutto / mies.,"{'x': 18.9641781, 'y': 50.1357681, 'addr:stree..."
39442,"https://www.pracuj.pl/praca/monter-reda,oferta...",praca stacjonarna,NaN,pracownik fizyczny,pełny etat,umowa o pracę,"['Wejherowska 108, Reda', 'Reda, pomorskie']",https://www.google.pl/maps/search/54.603878021...,['diagnozowanie oraz naprawa urządzeń klimatyz...,['doświadczenie na podobnym stanowisku (3 lat)...,"['pracę w dynamicznie rozwijającej się firmie,...",FIRMA ACTIVE CLIMA\nSpecjalizujemy się w nowoc...,"30,00‐40,00 zł brutto / godz.","30,00‐40,00 zł brutto / godz.","{'x': '18.30394172668457', 'y': '54.6038780212..."
39443,https://www.pracuj.pl/praca/doradca-ds-nieruch...,praca hybrydowa,NaN,"specjalista (mid / regular), starszy specjalis...",pełny etat,"umowa zlecenie, kontrakt b2b","['Marszałkowska 87, Warszawa', 'Warszawa, mazo...",https://www.google.pl/maps/search/52.227344512...,"['Praca w otwartym, pozytywnym zespole.', 'Ela...","['Kogo szukamy?', 'osób energicznych i otwarty...","['pracę w elastycznych godzinach, możliwych do...",Firma działająca na rynku nieruchomości poszuk...,UMOWA ZLECENIE\n5 000‐20 000 zł brutto / mies....,UMOWA ZLECENIE\n5 000‐20 000 zł brutto / mies....,"{'x': '21.012590408325195', 'y':

### Praca w polsce - markery

In [14]:
offers_df_country_markers = offers_df.filter(['open_street_raw'], axis=1)

In [15]:
offers_df_country_markers['open_street_raw'] = offers_df_country_markers.apply(lambda row: ast.literal_eval(row['open_street_raw']), axis=1)

In [16]:
offers_df_country_markers['marker_coords'] = \
    offers_df_country_markers.apply(lambda row: [float(row['open_street_raw']['lat']), float(row['open_street_raw']['lon'])], axis=1)

In [88]:
pracuj_df_country_markers = pracuj_df.filter(['inferred_location'], axis=1)

In [89]:
pracuj_df_country_markers = pracuj_df_country_markers.dropna().copy()

In [90]:
pracuj_df_country_markers['inferred_location'] = pracuj_df_country_markers.apply(lambda row: ast.literal_eval(row['inferred_location']), axis=1)

In [94]:
pracuj_df_country_markers['marker_coords'] = \
    pracuj_df_country_markers.apply(lambda row: [float(row['inferred_location']['y']), float(row['inferred_location']['x'])], axis=1)

In [98]:
df_country_markers = pd.concat([offers_df_country_markers, pracuj_df_country_markers], ignore_index=True)

In [22]:
START_LAT = 51.8512195
START_LON = 20.8417665
START_ZOOM = 6

In [99]:
map_country_markers = folium.Map(location=[START_LAT, START_LON], zoom_start=START_ZOOM)

In [100]:
for index, row in df_country_markers.iterrows():
    folium.CircleMarker(
        location=row['marker_coords'],
        tooltip=row['marker_coords'],
        radius=2,
#         weight=1,
        color="crimson",
        fill=False,
    ).add_to(map_country_markers)

In [101]:
map_country_markers.save("map_country_markers.html")

### Dalsza obróbka datasetu z markerami

In [102]:
df_markers = df_country_markers.copy()

In [110]:
geolocator = Nominatim(user_agent="http")
df_markers['geopy_address'] = \
    df_markers.apply(lambda row: geolocator.reverse(row['marker_coords']).address.split(', '), axis=1)

In [24]:
# df_wroclaw_markers.to_csv('markers_with_adressess.csv', index=True)
df_markers = pd.read_csv('markers_with_adressess.csv')

In [25]:
df_markers.drop_duplicates(subset=['open_street_raw', 'marker_coords', 'inferred_location', 'geopy_address'], inplace=True)

In [26]:
df_markers['geopy_address'] = df_markers.apply(lambda row: ast.literal_eval(row['geopy_address']), axis=1)
df_markers['marker_coords'] = df_markers.apply(lambda row: ast.literal_eval(row['marker_coords']), axis=1)

In [27]:
df_markers['geopy_address_len'] = df_markers.apply(lambda row: len(row['geopy_address']), axis=1)

In [28]:
df_markers.drop(df_markers[df_markers['geopy_address_len'] <= 4].index, inplace=True)
df_markers.drop(['geopy_address_len'], inplace = True, axis=1)
df_markers.drop(['Unnamed: 0'], inplace = True, axis=1)

In [29]:
df_markers['country'] = df_markers.apply(lambda row: row['geopy_address'][-1], axis=1)
df_markers.drop(df_markers[df_markers['country'] != 'Polska'].index, inplace=True)
df_markers.drop(['country'], inplace = True, axis=1)

In [30]:
df_markers['voivodeship'] = df_markers.apply(lambda row: row['geopy_address'][-3], axis=1)

In [31]:
df_markers['state'] = df_markers.apply(lambda row: row['geopy_address'][-4], axis=1)

In [32]:
df_markers['county'] = df_markers.apply(lambda row: row['geopy_address'][-5], axis=1)

### Praca we wrocławiu - markery

In [329]:
df_wroclaw_markers = df_markers[df_markers['state'] == 'Wrocław'].copy()

In [330]:
df_wroclaw_markers['county'] = df_wroclaw_markers.apply(lambda row: row['county'].replace("Osiedle ", "", 1), axis=1)

In [331]:
df_wroclaw_markers['count'] = df_wroclaw_markers.apply(lambda row: 1, axis=1)

In [332]:
df_wroclaw_markers_count = df_wroclaw_markers.groupby(['county'])['count'].sum().reset_index()

In [333]:
df_wroclaw_markers_count['county'] = df_wroclaw_markers_count.apply(lambda row: row['county'].replace("-", " - "), axis=1)

In [338]:
df_wroclaw_markers_count.drop(df_wroclaw_markers_count[df_wroclaw_markers_count['county'] == 'Stare Miasto'].index, inplace=True)

In [339]:
df_wroclaw_markers_count['count_norm'] = \
    (df_wroclaw_markers_count['count']-df_wroclaw_markers_count['count'].min())/(df_wroclaw_markers_count['count'].max()-df_wroclaw_markers_count['count'].min())

In [343]:
WROC_START_LAT = 51.13694038154205 
WROC_START_LON = 17.04576517482033
WROC_START_ZOOM = 8

wroclaw_geo = geopandas.read_file('D:\Studia\Semestr IX\Przetwarzanie Danych Przestrzennych\geo_maps\miasta\wroclaw-max.geojson')

wroclaw_map = folium.Map(location=[WROC_START_LAT, WROC_START_LON], zoom_start=WROC_START_ZOOM)

folium.Choropleth(
    geo_data=wroclaw_geo,
    name="choropleth",
    data=df_wroclaw_markers_count,
    columns=["county", "count_norm"],
    key_on="feature.properties.osiedle",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Jobs count",
).add_to(wroclaw_map)

folium.LayerControl().add_to(wroclaw_map)

for index, row in df_wroclaw_markers.iterrows():
    folium.CircleMarker(
        location=row['marker_coords'],
        tooltip=row['marker_coords'],
        radius=2,
#         weight=1,
        color="crimson",
        fill=False,
    ).add_to(wroclaw_map)


In [391]:
wroclaw_map.save("map_wroclaw_markers.html")

### Liczba ogłoszeń o pracę - województwa

In [379]:
df_voivodeship_markers = df_markers.copy()

In [380]:
df_voivodeship_markers.drop(df_voivodeship_markers[~df_voivodeship_markers['voivodeship'].str.contains('województwo')].index, inplace=True)

In [381]:
df_voivodeship_markers['voivodeship'] = df_voivodeship_markers.apply(lambda row: row['voivodeship'].replace("województwo ", "", 1), axis=1)

In [382]:
df_voivodeship_markers['count'] = df_voivodeship_markers.apply(lambda row: 1, axis=1)

In [386]:
df_voivodeship_markers_count = df_voivodeship_markers.groupby(['voivodeship'])['count'].sum().reset_index()

In [394]:
voivodeship_geo = geopandas.read_file('D:\Studia\Semestr IX\Przetwarzanie Danych Przestrzennych\geo_maps\wojewodztwa\wojewodztwa-max.geojson')

voivodeship_map = folium.Map(location=[START_LAT, START_LON], zoom_start=START_ZOOM)

folium.Choropleth(
    geo_data=voivodeship_geo,
    name="choropleth",
    data=df_voivodeship_markers_count,
    columns=["voivodeship", "count"],
    key_on="feature.properties.nazwa",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Jobs count",
).add_to(voivodeship_map)

folium.LayerControl().add_to(voivodeship_map)

for index, row in df_voivodeship_markers.iterrows():
    folium.CircleMarker(
        location=row['marker_coords'],
        tooltip=row['marker_coords'],
        radius=0.5,
#         weight=1,
        color="crimson",
        fill=False,
    ).add_to(voivodeship_map)

In [395]:
voivodeship_map.save("map_voivodeship_markers.html")

### Liczba ogłoszeń o pracę - powiaty

In [492]:
df_state_markers = df_markers.copy()

In [493]:
df_state_markers.drop(df_state_markers[~df_state_markers['voivodeship'].str.contains('województwo')].index, inplace=True)

In [451]:
state_geo = geopandas.read_file('D:\Studia\Semestr IX\Przetwarzanie Danych Przestrzennych\geo_maps\powiaty\powiaty-max.geojson')

In [494]:
def get_state_name(row):
    current_point = Point(row['marker_coords'][1], row['marker_coords'][0])
    containing_series = state_geo.contains(current_point)
    containing_series_bools = containing_series[containing_series == True]
    
    if containing_series_bools.empty:
        state = 'None'
    else:
        state = state_geo[['id', 'nazwa']].iloc[containing_series_bools.index.item()]['nazwa']
    return state

In [486]:
current_point = Point(18.994487141198455, 50.25972211320963)
current_point = Point(17.847767227493087, 49.68307698890241)


d = state_geo.contains(current_point)

In [495]:
df_state_markers['state'] = df_state_markers.apply(lambda row: get_state_name(row), axis=1)

In [496]:
df_state_markers['count'] = df_state_markers.apply(lambda row: 1, axis=1)

In [498]:
df_state_markers[df_state_markers['state'] == 'None']

,open_street_raw,marker_coords,inferred_location,geopy_address,voivodeship,state,county,count
37526,NaN,"[54.61025975, 18.514110303450295]","{'x': 18.514110303450295, 'y': 54.61025975, 'a...","[Molo w Mechelinkach, Nadmorska, Mechelinki, g...",województwo pomorskie,None,gmina Kosakowo,1


In [499]:
df_state_markers_count = df_state_markers.groupby(['state'])['count'].sum().reset_index()

In [503]:
df_state_markers_count.drop(df_state_markers_count[df_state_markers_count['state'] == 'None'].index, inplace=True)

In [509]:
df_state_markers_count.drop(df_state_markers_count[df_state_markers_count['state'] == 'powiat Warszawa'].index, inplace=True)

In [510]:
# state_geo = geopandas.read_file('D:\Studia\Semestr IX\Przetwarzanie Danych Przestrzennych\ms_A02_Granice_powiatow.gml')
# state_geo = geopandas.read_file('D:\Studia\Semestr IX\Przetwarzanie Danych Przestrzennych\geo_maps\powiaty\powiaty-max.geojson')

state_map = folium.Map(location=[START_LAT, START_LON], zoom_start=START_ZOOM)

folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=df_state_markers_count,
    columns=["state", "count"],
    key_on="feature.properties.nazwa",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Jobs count",
).add_to(state_map)

folium.LayerControl().add_to(state_map)

for index, row in df_state_markers.iterrows():
    folium.CircleMarker(
        location=row['marker_coords'],
        tooltip=row['marker_coords'],
        radius=0.5,
#         weight=1,
        color="crimson",
        fill=False,
    ).add_to(state_map)

In [511]:
state_map.save("map_state_markers.html")

### Bezrobocie - polska

In [201]:
df_unemployment_country = pd.read_pickle('all_data\df_bezrobocie_polska.pkl')
df_unemployment_voivodeship = pd.read_pickle('all_data\df_bezrobocie_wojew.pkl')
df_unemployment_state = pd.read_pickle('all_data\df_bezrobocie_powiat.pkl')

In [202]:
df_unemployment_voivodeship['unit'] = df_unemployment_voivodeship.apply(lambda row: row['unit'].replace(" ", ""), axis=1)

In [203]:
df_unemployment_voivodeship['unit'] = df_unemployment_voivodeship.apply(lambda row: row['unit'].replace("Woj. ", "", 1).lower(), axis=1)

In [204]:
df_unemployment_state['unit'] = df_unemployment_state.apply(lambda row: row['unit'].replace(" ", ""), axis=1)

In [205]:
LANGS = [
    'ĄąĆćĘęŁłŃńÓóŚśŹźŻż',               # Polish
    ]
LANGS_ = [
    'ĄĆĘŁŃÓŚŹŻ',               # Polish
    ]

pattern = '[A-Za-z{langs}][^A-Z{langs_}]*'.format(langs=''.join(LANGS), langs_=''.join(LANGS_))
pattern = re.compile(pattern)

df_unemployment_state['unit'] = \
    df_unemployment_state.apply(lambda row: ' '.join(pattern.findall(row['unit'])), axis=1)

In [206]:
df_unemployment_state['unit'] = df_unemployment_state.apply(lambda row: 'powiat ' + row['unit'], axis=1)

In [207]:
df_unemployment_state['unit'] = df_unemployment_state.apply(lambda row: row['unit'].replace("m. ", "", 1), axis=1)
df_unemployment_state['unit'] = df_unemployment_state.apply(lambda row: row['unit'].replace("m.", "", 1), axis=1)

In [212]:
df_unemployment_state['unit'] = df_unemployment_state.apply(lambda row: row['unit'].replace("warszawskizachodni", "warszawski zachodni", 1), axis=1)
df_unemployment_state['unit'] = df_unemployment_state.apply(lambda row: row['unit'].replace("łódzkiwschodni", "łódzki wschodni", 1), axis=1)
df_unemployment_state['unit'] = df_unemployment_state.apply(lambda row: row['unit'].replace("karkonoski", "jeleniogórski", 1), axis=1)
df_unemployment_state['unit'] = df_unemployment_state.apply(lambda row: row['unit'].replace("Jastrzębie- Zdrój", "Jastrzębie-Zdrój", 1), axis=1)
df_unemployment_state['unit'] = df_unemployment_state.apply(lambda row: row['unit'].replace("Bielsko- Biała", "Bielsko-Biała", 1), axis=1)

In [213]:
df_unemployment_state[df_unemployment_state['unit'].str.contains('Biel')]['unit']

283    powiat Bielsko-Biała
Name: unit, dtype: object

### Bezrobocie - województwa

In [41]:
voivodeship_geo = geopandas.read_file('D:\Studia\Semestr IX\Przetwarzanie Danych Przestrzennych\geo_maps\wojewodztwa\wojewodztwa-max.geojson')

voivodeship_map = folium.Map(location=[START_LAT, START_LON], zoom_start=START_ZOOM)

folium.Choropleth(
    geo_data=voivodeship_geo,
    name="choropleth",
    data=df_unemployment_voivodeship,
    columns=["unit", "rate_p"],
    key_on="feature.properties.nazwa",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Unemployment rate",
).add_to(voivodeship_map)

folium.LayerControl().add_to(voivodeship_map)

In [42]:
voivodeship_map.save("map_voivodeship_unemployment.html")

### Bezrobocie - powiaty

In [214]:
state_geo = geopandas.read_file('D:\Studia\Semestr IX\Przetwarzanie Danych Przestrzennych\geo_maps\powiaty\powiaty-max.geojson')

state_map = folium.Map(location=[START_LAT, START_LON], zoom_start=START_ZOOM)

folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=df_unemployment_state,
    columns=["unit", "rate_p"],
    key_on="feature.properties.nazwa",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Unemployment rate",
).add_to(state_map)

folium.LayerControl().add_to(state_map)

In [216]:
state_map.save("map_state_unemployment.html")

### Liczenie indeksu Giniego - test

In [33]:
df_gini_idx = df_markers.copy()

In [34]:
df_gini_idx.drop(df_gini_idx[~df_gini_idx['voivodeship'].str.contains('województwo')].index, inplace=True)

In [35]:
state_geo = geopandas.read_file('D:\Studia\Semestr IX\Przetwarzanie Danych Przestrzennych\geo_maps\powiaty\powiaty-max.geojson')

In [36]:
def get_state_name(row):
    current_point = Point(row['marker_coords'][1], row['marker_coords'][0])
    containing_series = state_geo.contains(current_point)
    containing_series_bools = containing_series[containing_series == True]
    
    if containing_series_bools.empty:
        state = 'None'
    else:
        state = state_geo[['id', 'nazwa']].iloc[containing_series_bools.index.item()]['nazwa']
    return state

In [37]:
df_gini_idx['state'] = df_gini_idx.apply(lambda row: get_state_name(row), axis=1)

In [39]:
df_gini_idx['wealth'] = df_gini_idx.apply(lambda row: round(np.random.uniform(1600,7000), 2), axis=1)

In [127]:
df_states_names = pd.read_csv('powiaty.csv')

In [134]:
gini_indices = []
for index, row in df_states_names.iterrows():
    state_name = row['state_name']
    df_state = df_gini_idx[df_gini_idx['state'] == state_name].copy()
    df_state.sort_values('wealth', ascending=True, inplace=True)
    df_state_mean = df_state.groupby(['state'])['wealth'].mean().reset_index()
    state_mean = df_state_mean['wealth'].iloc[0]
    n_records = len(df_state.index)
    
    gini_idx = 0.
    for i, (index, state_row) in enumerate(df_state.iterrows()):
        gini_idx += (2 * (i + 1) - n_records - 1) * state_row['wealth']
    
    gini_idx /= n_records**2 * state_mean
    gini_indices.append(gini_idx)

In [135]:
df_state_gini = df_states_names.copy()
series_gini_indices = pd.Series(gini_indices)
df_state_gini['gini_idx'] = series_gini_indices.values

In [139]:
START_LAT = 51.8512195
START_LON = 20.8417665
START_ZOOM = 6

state_geo = geopandas.read_file('D:\Studia\Semestr IX\Przetwarzanie Danych Przestrzennych\geo_maps\powiaty\powiaty-max.geojson')

gini_map = folium.Map(location=[START_LAT, START_LON], zoom_start=START_ZOOM)

folium.Choropleth(
    geo_data=state_geo,
    name="gini_idx",
    data=df_state_gini,
    columns=["state_name", "gini_idx"],
    key_on="feature.properties.nazwa",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Gini Coefficient",
).add_to(gini_map)

folium.LayerControl().add_to(gini_map)

In [140]:
gini_map.save("map_gini_index.html")

### Liczenie indeksu Giniego - powiaty

In [144]:
df_gini_idx = pd.read_csv('pracuj_powiaty_wojew_extracted_salary.csv')

In [146]:
df_states_names = pd.read_csv('powiaty.csv')

In [150]:
gini_indices = []
for index, row in df_states_names.iterrows():
    state_name = row['state_name']
    df_state = df_gini_idx[df_gini_idx['state'] == state_name].copy()
    df_state.sort_values('salary_inferred', ascending=True, inplace=True)
    df_state_mean = df_state.groupby(['state'])['salary_inferred'].mean().reset_index()
    if len(df_state_mean) != 0:
        state_mean = df_state_mean['salary_inferred'].iloc[0]
        n_records = len(df_state.index)

        gini_idx = 0.
        for i, (index, state_row) in enumerate(df_state.iterrows()):
            gini_idx += (2 * (i + 1) - n_records - 1) * state_row['salary_inferred']

        gini_idx /= n_records**2 * state_mean
        gini_indices.append(gini_idx)
    else:
        gini_indices.append(-1.)

In [151]:
df_state_gini = df_states_names.copy()
series_gini_indices = pd.Series(gini_indices)
df_state_gini['gini_idx'] = series_gini_indices.values

In [153]:
START_LAT = 51.8512195
START_LON = 20.8417665
START_ZOOM = 6

state_geo = geopandas.read_file('D:\Studia\Semestr IX\Przetwarzanie Danych Przestrzennych\geo_maps\powiaty\powiaty-max.geojson')

gini_map = folium.Map(location=[START_LAT, START_LON], zoom_start=START_ZOOM)

folium.Choropleth(
    geo_data=state_geo,
    name="gini_idx",
    data=df_state_gini,
    columns=["state_name", "gini_idx"],
    key_on="feature.properties.nazwa",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Gini Coefficient",
).add_to(gini_map)

folium.LayerControl().add_to(gini_map)

In [154]:
gini_map.save("map_gini_index_states.html")

### Liczenie indeksu Giniego - województwa

In [144]:
df_gini_idx = pd.read_csv('pracuj_powiaty_wojew_extracted_salary.csv')

In [156]:
df_gini_idx.head(1)

,url,work_modes,remote,employment_type,schedule,contract,locations,location_url,duties,requirements,...,about_us,salary,benefits,inferred_location,marker_coords,geopy_address,voivodeship,state,county,salary_inferred
0,https://www.pracuj.pl/praca/frontend-developer...,"praca zdalna, praca hybrydowa",rekrutacja zdalna,specjalista (mid / regular),pełny etat,"umowa o pracę, kontrakt b2b","['aleja Słowiańska 10B, Bielany, Warszawa', 'W...",https://www.google.pl/maps/search/52.276634216...,[],['wykształcenie wyższe techniczne lub równoważ...,...,BMS sp. z o.o.\r\nOd ponad 20 lat tworzymy zaa...,UMOWA O PRACĘ\r\n6 700‐13 400 zł brutto / mies...,UMOWA O PRACĘ\r\n6 700‐13 400 zł brutto / mies...,"{'x': '20.967201232910156', 'y': '52.276634216...","[52.276634216308594, 20.967201232910156]","['10A', 'Aleja Słowiańska', 'Marymont-Kaskada'...",województwo mazowieckie,powiat Warszawa,Bielany,10050.0


In [176]:
df_voivodeships_names = pd.read_csv('wojewodztwa.csv')

In [178]:
gini_indices = []
for index, row in df_voivodeships_names.iterrows():
    voivodeship_name = row['voivodeship_name']
    df_voivodeship = df_gini_idx[df_gini_idx['voivodeship'] == voivodeship_name].copy()
    df_voivodeship.sort_values('salary_inferred', ascending=True, inplace=True)
    df_voivodeship_mean = df_voivodeship.groupby(['voivodeship'])['salary_inferred'].mean().reset_index()
    if len(df_voivodeship_mean) != 0:
        voivodeship_mean = df_voivodeship_mean['salary_inferred'].iloc[0]
        n_records = len(df_voivodeship.index)

        gini_idx = 0.
        for i, (index, voivodeship_row) in enumerate(df_voivodeship.iterrows()):
            gini_idx += (2 * (i + 1) - n_records - 1) * voivodeship_row['salary_inferred']

        gini_idx /= n_records**2 * voivodeship_mean
        gini_indices.append(gini_idx)
    else:
        gini_indices.append(-1.)

In [182]:
df_state_gini = df_voivodeships_names.copy()
series_gini_indices = pd.Series(gini_indices)
df_state_gini['gini_idx'] = series_gini_indices.values

In [184]:
df_state_gini['voivodeship_name'] = df_state_gini.apply(lambda row: row['voivodeship_name'].replace("województwo ", "", 1), axis=1)

In [185]:
START_LAT = 51.8512195
START_LON = 20.8417665
START_ZOOM = 6

voivodeship_geo = geopandas.read_file('D:\Studia\Semestr IX\Przetwarzanie Danych Przestrzennych\geo_maps\wojewodztwa\wojewodztwa-max.geojson')

gini_map = folium.Map(location=[START_LAT, START_LON], zoom_start=START_ZOOM)

folium.Choropleth(
    geo_data=voivodeship_geo,
    name="gini_idx",
    data=df_state_gini,
    columns=["voivodeship_name", "gini_idx"],
    key_on="feature.properties.nazwa",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Gini Coefficient",
).add_to(gini_map)

folium.LayerControl().add_to(gini_map)

In [186]:
gini_map.save("map_gini_index_voivodeships.html")

### Pracuj.pl - zarobki w powiatach

In [99]:
df = pd.read_csv('all_data\pracuj_inferred_location.csv')
df = df[df['salary'].notna()]
df = df[df['inferred_location'].notna()]

In [100]:
df_geodata = df.copy()

In [102]:
df_geodata['inferred_location'] = df_geodata.apply(lambda row: ast.literal_eval(row['inferred_location']), axis=1)

In [105]:
df_geodata['marker_coords'] = \
    df_geodata.apply(lambda row: [float(row['inferred_location']['y']), float(row['inferred_location']['x'])], axis=1)

In [107]:
geolocator = Nominatim(user_agent="http")
df_geodata['geopy_address'] = \
    df_geodata.apply(lambda row: geolocator.reverse(row['marker_coords']).address.split(', '), axis=1)

In [109]:
df_geodata.to_csv('pracuj_powiaty.csv', index=False)

In [125]:
len(df_geodata)

4465

In [112]:
df_geodata['geopy_address_len'] = df_geodata.apply(lambda row: len(row['geopy_address']), axis=1)

In [113]:
df_geodata.drop(df_geodata[df_geodata['geopy_address_len'] <= 4].index, inplace=True)
df_geodata.drop(['geopy_address_len'], inplace = True, axis=1)
# df_geodata.drop(['Unnamed: 0'], inplace = True, axis=1)

In [115]:
df_geodata['country'] = df_geodata.apply(lambda row: row['geopy_address'][-1], axis=1)
df_geodata.drop(df_geodata[df_geodata['country'] != 'Polska'].index, inplace=True)
df_geodata.drop(['country'], inplace = True, axis=1)

In [117]:
df_geodata['voivodeship'] = df_geodata.apply(lambda row: row['geopy_address'][-3], axis=1)

In [118]:
df_geodata['state'] = df_geodata.apply(lambda row: row['geopy_address'][-4], axis=1)

In [119]:
df_geodata['county'] = df_geodata.apply(lambda row: row['geopy_address'][-5], axis=1)

In [120]:
df_geodata.drop(df_geodata[~df_geodata['voivodeship'].str.contains('województwo')].index, inplace=True)

In [122]:
state_geo = geopandas.read_file('D:\Studia\Semestr IX\Przetwarzanie Danych Przestrzennych\geo_maps\powiaty\powiaty-max.geojson')

In [123]:
def get_state_name(row):
    current_point = Point(row['marker_coords'][1], row['marker_coords'][0])
    containing_series = state_geo.contains(current_point)
    containing_series_bools = containing_series[containing_series == True]
    
    if containing_series_bools.empty:
        state = 'None'
    else:
        state = state_geo[['id', 'nazwa']].iloc[containing_series_bools.index.item()]['nazwa']
    return state

In [124]:
df_geodata['state'] = df_geodata.apply(lambda row: get_state_name(row), axis=1)

In [126]:
df_geodata.to_csv('pracuj_powiaty_wojew_final.csv', index=False)